## 高分辨率图像修复与增强（SR/Restoration GAN）

基于 MindSpore 框架实现图像超分辨率重建与修复
项目目标
- 低清图像的超分辨率重建（LR→HR，×4放大）
- 去噪、去模糊、去压缩伪影
- 真实旧照片修复与增强

技术路线
- **基线模型**：SRResNet（像素损失训练，稳定易训）
- **进阶模型**：ESRGAN（RRDBNet + RaGAN + 感知损失，感知质量更好）

评估指标
- 有GT：PSNR↑、SSIM↑
- 无GT：主观评价

## 0. 安装模块依赖

#### 先自行创建一个python3.9的虚拟环境，并在此环境下安装依赖


In [ ]:
# 安装python依赖包（建议在终端下载依赖，后续操作用到哪个装哪个，不要一次性全部安装，会出现依赖版本冲突）
# pip install opencv-python Pillow matplotlib scikit-image tqdm

### 0.1 MindSpore 2.2.14（CPU版） 安装命令
#### 从官网下载安装wheel包进行安装，避免使用pip直接安装导致的依赖冲突问题

选择对应的操作系统和python版本(当前文档使用的版本为windos的CPU版mindspore): mindspore-2.2.14-cp39-cp39-win_amd64.whl

下载完whell包之后将.whl文件放到项目根目录下。

随后执行以下命令(后续所有的终端命令请自行复制到终端运行)

In [ ]:
# 安装 MindSpore（适用于 Windows 和 Python 3.9）
# python -m pip install mindspore-2.2.14-cp39-cp39-win_amd64.whl --no-cache-dir

### 0.2 MindSpore 2.2.14（GPU版） 安装命令，适用于CUDA 11.6 
GPU版详细安装命令请参考官网：https://gitee.com/mindspore/docs/blob/r2.2/install/mindspore_gpu_install_pip.md

GPU版本安装前需要先安装对应版本的CUDA和CUDNN，建议参考官网说明进行安装，或者使用以下命令安装（不会安装可以直接询问AI）

若未安装CUDA11.6和cuDNN8.5.0，请先使用以下命令安装(请复制到终端安装)

PS：Mindspore GPU版本只适用于Linux系统，本文档环境为 Ubuntu 20.04系统（云服务器）

需要使用云服务器可以选择AutoDL，如何使用建议参考B站视频：https://www.bilibili.com/video/BV1xDCbYCEZo/?spm_id_from=333.337.search-card.all.click&vd_source=b424755df5875d2c243ba347d8c91dcb

云服务器默认提供了CUDA11.6和cuDNN8.5.0，无需重复安装，直接执行mindspore安装命令即可。

In [ ]:
# 云服务器默认提供CUDA 11.6，无需额外安装，若当前云服务器未提供CUDA 11.6，可参考以下步骤安装
# 安装CUDA 11.6
# 1. 下载CUDA 11.6的deb安装包（约3GB，确保网络通畅）
# wget https://developer.download.nvidia.com/compute/cuda/11.6.0/local_installers/cuda-repo-ubuntu2004-11-6-local_11.6.0-510.39.01-1_amd64.deb

# 2. 安装deb包并导入公钥
# sudo dpkg -i cuda-repo-ubuntu2004-11-6-local_11.6.0-510.39.01-1_amd64.deb
# sudo cp /var/cuda-repo-ubuntu2004-11-6-local/cuda-*-keyring.gpg /usr/share/keyrings/

# 3. 更新软件源并安装CUDA 11.6（包含驱动，若已装兼容驱动可跳过驱动安装）
# sudo apt-get update
# sudo apt-get install -y cuda-toolkit-11-6  # 仅安装CUDA工具包（推荐）

# 编辑用户环境变量文件（bash用户用.bashrc，zsh用户用.zshrc）
# nano ~/.bashrc

# 在文件末尾添加以下3行（复制粘贴）
# export PATH=/usr/local/cuda-11.6/bin:$PATH
# export LD_LIBRARY_PATH=/usr/local/cuda-11.6/lib64:$LD_LIBRARY_PATH
# export CUDA_HOME=/usr/local/cuda-11.6

# 保存生效（按 Ctrl+O 保存，Ctrl+X 退出nano）
# source ~/.bashrc

# 查看CUDA版本（应输出11.6）
# nvcc -V  # 注意是大写V

# 安装cuDNN 8.5.0
# 访问 NVIDIA 官网下载页：https://developer.nvidia.com/cudnn-archive
# 找到 Download cuDNN v8.5.0 (August 8th, 2022), for CUDA 11.x

# 下载 Local Installer for Linux x86_64 (Tar)到本地后续上传到Linux服务器，假设文件名为cudnn-linux-x86_64-8.5.0.96_cuda11-archive.tar.xz
# tar -xJvf cudnn-linux-x86_64-8.5.0.96_cuda11-archive.tar.xz
# 进入解压后的cuDNN目录（替换为你实际的解压目录名）
# cd cudnn-linux-x86_64-8.5.0.96_cuda11-archive

# 复制头文件到CUDA的include目录
# sudo cp include/cudnn*.h /usr/local/cuda-11.6/include/

# 复制库文件到CUDA的lib64目录（-P保留符号链接）
# sudo cp lib64/libcudnn* /usr/local/cuda-11.6/lib64/

# 给复制的文件添加全局读取权限，避免后续程序无法调用 cuDNN
# sudo chmod a+r /usr/local/cuda-11.6/include/cudnn*.h /usr/local/cuda-11.6/lib64/libcudnn*

# 更新动态链接库缓存
# sudo ldconfig /usr/local/cuda-11.6/lib64

# 查看cuDNN的版本（从头文件中读取）
# cat /usr/local/cuda-11.6/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

# 若输出类似以下内容，说明安装成功：
#define CUDNN_MAJOR 8
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0

In [ ]:
# 下载MindSpore 2.2.14 whl包
# wget https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl

# 安装MindSpore
# pip install mindspore-2.2.14-cp39-cp39-linux_x86_64.whl

---
## 1. 环境搭建与验证

In [ ]:
# 环境验证与GPU诊断
import mindspore
from mindspore import context
import os
import subprocess

print("=" * 60)
print("MindSpore环境诊断 - 图像修复与增强项目")
print("=" * 60)

# 1. MindSpore版本信息
print(f"\n[MindSpore信息]")
print(f"   版本: {mindspore.__version__}")
print(f"   安装路径: {mindspore.__file__}")

# 2. 检查CUDA环境
print(f"\n[CUDA环境检查]")
cuda_available = False
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("✅ NVIDIA驱动已安装")
        lines = result.stdout.split('\n')
        for line in lines:
            if 'CUDA Version' in line or 'GeForce' in line or 'RTX' in line:
                print(f"   {line.strip()}")
        cuda_available = True
    else:
        print("❌ NVIDIA驱动未找到")
except Exception as e:
    print(f"❌ 无法检测NVIDIA驱动: {e}")

# 3. MindSpore设备支持检查
print(f"\n[MindSpore设备支持]")
gpu_supported = False
try:
    context.set_context(device_target="GPU")
    print(f"✅ MindSpore支持GPU")
    gpu_supported = True
except Exception as e:
    print(f"❌ MindSpore不支持GPU，使用CPU模式")
    context.set_context(device_target="CPU")

# 4. 当前设备状态
print(f"\n[当前设备状态]")
current_device = context.get_context('device_target')
print(f"   使用设备: {current_device}")

# 5. 总结
print("\n" + "=" * 60)
print("诊断总结")
print("=" * 60)
if gpu_supported and cuda_available:
    print("✅ GPU环境完整，可以使用GPU加速训练")
else:
    print("ℹ️  当前环境使用CPU模式（训练速度较慢但功能完整）")
print("=" * 60)

---
## 2. 数据集加载与预处理

数据目录结构：
```
./data/
  train_HR/    # 训练高分辨率图像（Ground Truth）
  train_LR/    # 训练低分辨率/劣化图像（与HR配对）
  val_HR/      # 验证高分辨率图像
  val_LR/      # 验证低分辨率图像
  old_photos/  # 真实旧照片（无GT，用于推理展示）
```

In [ ]:
# 数据集统计与检查
import os
from PIL import Image
import numpy as np

# 配置
DATA_ROOT = "./data"
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp')

def count_images(directory):
    """统计目录下的图片数量"""
    if not os.path.exists(directory):
        return 0, []
    files = [f for f in os.listdir(directory) 
             if f.lower().endswith(IMAGE_EXTENSIONS) and os.path.isfile(os.path.join(directory, f))]
    return len(files), sorted(files)

# 统计各数据集
print("=" * 60)
print("数据集统计")
print("=" * 60)

datasets = {
    'train_HR': os.path.join(DATA_ROOT, 'train_HR'),
    'train_LR': os.path.join(DATA_ROOT, 'train_LR'),
    'val_HR': os.path.join(DATA_ROOT, 'val_HR'),
    'val_LR': os.path.join(DATA_ROOT, 'val_LR'),
    'old_photos': os.path.join(DATA_ROOT, 'old_photos')
}

for name, path in datasets.items():
    count, files = count_images(path)
    print(f"{name}: {count} 张图像")
    if count > 0:
        print(f"   示例: {files[:3]}")
    print("-" * 40)

In [ ]:
# 成对数据集加载器（内存预加载版本 + 数据增强）
import os
import mindspore.dataset as ds
import mindspore.dataset.vision as vision
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# ===================== 数据配置参数（平衡版：GPU利用率 + 训练速度） =====================
DATA_ROOT = "./data"
SCALE = 4  # 超分辨率放大倍数
HR_SIZE = 192  # HR patch尺寸（保持192，避免过长训练时间）
LR_SIZE = HR_SIZE // SCALE  # LR patch尺寸
BATCH_SIZE = 16  # 批大小
NUM_WORKERS = 1  # 内存预加载模式下设为1即可
MAX_TRAIN_SAMPLES = None  # 训练集最大加载数量，None表示加载全部
MAX_VAL_SAMPLES = None    # 验证集最大加载数量，None表示加载全部
USE_AUGMENTATION = True   # 是否启用数据增强

# 不使用梯度累积（节省训练时间）
ACCUMULATION_STEPS = 1    # 设为1表示不累积
# ====================================================================================

plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

def get_paired_paths(hr_dir, lr_dir, scale=4, max_samples=None):
    """获取配对的HR/LR图像路径
    
    支持处理文件名中包含空格的情况
    HR文件名格式：{stem}.{ext}（可能包含空格）
    LR文件名格式：{stem}x{scale}.png 或 {stem} x{scale}.png
    
    Args:
        hr_dir: HR图像目录
        lr_dir: LR图像目录
        scale: 放大倍数
        max_samples: 最大加载数量，None表示加载全部
    """
    hr_files = sorted([f for f in os.listdir(hr_dir) if f.lower().endswith(IMAGE_EXTENSIONS)])
    lr_files = os.listdir(lr_dir)
    paired = []
    
    for hr_name in hr_files:
        stem, ext = os.path.splitext(hr_name)
        hr_path = os.path.join(hr_dir, hr_name)
        
        # 尝试多种LR文件名格式
        lr_candidates = [
            f"{stem}x{scale}.png",           # 标准格式：0003x4.png
            f"{stem} x{scale}.png",          # 带空格：0001 x4.png
            f"{stem.strip()}x{scale}.png",   # 去除stem末尾空格
        ]
        
        lr_path = None
        for lr_name in lr_candidates:
            candidate_path = os.path.join(lr_dir, lr_name)
            if os.path.exists(candidate_path):
                lr_path = candidate_path
                break
        
        # 如果还没找到，尝试模糊匹配
        if lr_path is None:
            stem_clean = stem.strip()
            for f in lr_files:
                # 检查文件名是否包含stem和scale标记
                f_stem = os.path.splitext(f)[0]
                if f_stem.strip().replace(' ', '').startswith(stem_clean.replace(' ', '')) and f'x{scale}' in f:
                    lr_path = os.path.join(lr_dir, f)
                    break
        
        if lr_path and os.path.exists(lr_path):
            paired.append((hr_path, lr_path))
            # 达到最大数量时提前退出
            if max_samples is not None and len(paired) >= max_samples:
                break
        else:
            print(f"警告: 未找到配对LR: {hr_name} -> 尝试过: {lr_candidates[:2]}")
    
    return paired

def preprocess_image(img, target_size):
    """预处理单张图像：确保尺寸足够"""
    w, h = img.size
    if w < target_size or h < target_size:
        new_w = max(w, target_size)
        new_h = max(h, target_size)
        img = img.resize((new_w, new_h), Image.BICUBIC)
    return img

def augment_pair(hr_crop, lr_crop):
    """对HR和LR图像对同时应用数据增强
    
    增强方式（随机选择）：
    - 水平翻转 (50%概率)
    - 垂直翻转 (50%概率)
    - 旋转90°的倍数 (随机0/90/180/270度)
    
    注意：必须对HR和LR应用相同的变换以保持配对一致性
    """
    # 随机水平翻转
    if np.random.random() < 0.5:
        hr_crop = np.flip(hr_crop, axis=1).copy()
        lr_crop = np.flip(lr_crop, axis=1).copy()
    
    # 随机垂直翻转
    if np.random.random() < 0.5:
        hr_crop = np.flip(hr_crop, axis=0).copy()
        lr_crop = np.flip(lr_crop, axis=0).copy()
    
    # 随机旋转90°的倍数 (0, 1, 2, 3 分别对应 0°, 90°, 180°, 270°)
    k = np.random.randint(0, 4)
    if k > 0:
        hr_crop = np.rot90(hr_crop, k).copy()
        lr_crop = np.rot90(lr_crop, k).copy()
    
    return hr_crop, lr_crop

class PairedDataGenerator:
    """成对数据生成器（内存预加载版本 + 数据增强）
    
    在初始化时将所有图像加载到内存，训练时直接从内存读取，
    消除磁盘I/O延迟，大幅提升GPU利用率。
    
    数据增强：随机翻转 + 随机旋转，有效数据量增加8倍
    """
    def __init__(self, hr_dir, lr_dir, hr_size=128, scale=4, is_train=True, max_samples=None, use_augmentation=True):
        self.pairs = get_paired_paths(hr_dir, lr_dir, scale, max_samples)
        self.hr_size = hr_size
        self.lr_size = hr_size // scale
        self.scale = scale
        self.is_train = is_train
        self.use_augmentation = use_augmentation and is_train  # 只在训练时增强
        
        # 内存预加载所有图像
        self.hr_images = []
        self.lr_images = []
        
        if len(self.pairs) > 0:
            print(f"📥 预加载 {len(self.pairs)} 对图像到内存...")
            for hr_path, lr_path in tqdm(self.pairs, desc="加载图像", unit="张"):
                # 读取并预处理HR图像
                hr_img = Image.open(hr_path).convert('RGB')
                hr_img = preprocess_image(hr_img, hr_size)
                self.hr_images.append(np.array(hr_img))
                
                # 读取并预处理LR图像
                lr_img = Image.open(lr_path).convert('RGB')
                lr_img = preprocess_image(lr_img, self.lr_size)
                self.lr_images.append(np.array(lr_img))
            
            # 计算内存占用
            hr_mem = sum(img.nbytes for img in self.hr_images) / 1024 / 1024
            lr_mem = sum(img.nbytes for img in self.lr_images) / 1024 / 1024
            print(f"✅ 预加载完成！内存占用: HR={hr_mem:.1f}MB, LR={lr_mem:.1f}MB, 总计={hr_mem+lr_mem:.1f}MB")
            print(f"   示例配对: {os.path.basename(self.pairs[0][0])} <-> {os.path.basename(self.pairs[0][1])}")
            if self.use_augmentation:
                print(f"   🔄 数据增强已启用（有效数据量 ×8）")
    
    def __getitem__(self, index):
        hr_img = self.hr_images[index]
        lr_img = self.lr_images[index]
        
        hr_h, hr_w = hr_img.shape[:2]
        lr_h, lr_w = lr_img.shape[:2]
        
        # 随机裁剪位置（基于HR坐标）
        max_x = max(0, hr_w - self.hr_size)
        max_y = max(0, hr_h - self.hr_size)
        x = np.random.randint(0, max_x + 1) if max_x > 0 else 0
        y = np.random.randint(0, max_y + 1) if max_y > 0 else 0
        
        # 裁剪HR
        hr_crop = hr_img[y:y+self.hr_size, x:x+self.hr_size]
        
        # 对应位置裁剪LR（按比例缩放坐标）
        lx = min(x // self.scale, max(0, lr_w - self.lr_size))
        ly = min(y // self.scale, max(0, lr_h - self.lr_size))
        lr_crop = lr_img[ly:ly+self.lr_size, lx:lx+self.lr_size]
        
        # 数据增强（仅训练时）
        if self.use_augmentation:
            hr_crop, lr_crop = augment_pair(hr_crop, lr_crop)
        
        # 归一化到[-1, 1]并转换为CHW格式
        hr_np = hr_crop.astype(np.float32) / 127.5 - 1.0
        lr_np = lr_crop.astype(np.float32) / 127.5 - 1.0
        
        # HWC -> CHW
        hr_np = hr_np.transpose(2, 0, 1)
        lr_np = lr_np.transpose(2, 0, 1)
        
        return hr_np, lr_np
    
    def __len__(self):
        return len(self.pairs)

def create_sr_dataset(hr_dir, lr_dir, hr_size=128, scale=4, batch_size=4, is_train=True, max_samples=None, use_augmentation=True):
    """创建超分辨率数据集
    
    Args:
        hr_dir: HR图像目录
        lr_dir: LR图像目录
        hr_size: HR patch尺寸
        scale: 放大倍数
        batch_size: 批大小
        is_train: 是否为训练集
        max_samples: 最大加载数量，None表示加载全部
        use_augmentation: 是否启用数据增强
    
    注意: batch_size必须小于数据量，否则会因drop_remainder=True导致0个batch
    """
    generator = PairedDataGenerator(hr_dir, lr_dir, hr_size, scale, is_train, max_samples, use_augmentation)
    
    if len(generator) == 0:
        print(f"❌ 未找到配对数据！请检查目录: {hr_dir} 和 {lr_dir}")
        return None
    
    # 检查batch_size是否合理
    if batch_size > len(generator):
        print(f"⚠️ 警告: BATCH_SIZE({batch_size}) > 数据量({len(generator)})，自动调整为 {len(generator)}")
        batch_size = len(generator)
    
    dataset = ds.GeneratorDataset(
        source=generator,
        column_names=["hr", "lr"],
        shuffle=is_train,
        num_parallel_workers=NUM_WORKERS  # 内存预加载模式下设为1
    )
    
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

# 创建数据集
print("\n=== 创建数据集（内存预加载模式 + 数据增强） ===")
train_dataset = create_sr_dataset(
    os.path.join(DATA_ROOT, 'train_HR'),
    os.path.join(DATA_ROOT, 'train_LR'),
    hr_size=HR_SIZE, scale=SCALE, batch_size=BATCH_SIZE, is_train=True,
    max_samples=MAX_TRAIN_SAMPLES,  # 训练集加载数量限制
    use_augmentation=USE_AUGMENTATION  # 训练集启用数据增强
)

val_dataset = create_sr_dataset(
    os.path.join(DATA_ROOT, 'val_HR'),
    os.path.join(DATA_ROOT, 'val_LR'),
    hr_size=HR_SIZE, scale=SCALE, batch_size=1, is_train=False,
    max_samples=MAX_VAL_SAMPLES,  # 验证集加载数量限制
    use_augmentation=False  # 验证集不做增强
)

if train_dataset and val_dataset:
    print(f"\n训练集批次数: {train_dataset.get_dataset_size()}")
    print(f"验证集批次数: {val_dataset.get_dataset_size()}")
else:
    print("\n❌ 数据集创建失败，请检查数据目录")

In [ ]:
# 可视化数据样本
def denormalize(img):
    """反归一化: [-1,1] -> [0,255]"""
    img = (img * 127.5 + 127.5).clip(0, 255).astype(np.uint8)
    if img.shape[0] == 3:  # CHW -> HWC
        img = img.transpose(1, 2, 0)
    return img

# 获取一个batch进行可视化
try:
    sample = next(train_dataset.create_dict_iterator())
    hr_batch = sample['hr'].asnumpy()
    lr_batch = sample['lr'].asnumpy()
    
    print(f"HR shape: {hr_batch.shape}")
    print(f"LR shape: {lr_batch.shape}")
    
    # 可视化前2对
    n_show = min(2, hr_batch.shape[0])
    fig, axes = plt.subplots(n_show, 2, figsize=(10, 5*n_show))
    if n_show == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n_show):
        hr_img = denormalize(hr_batch[i])
        lr_img = denormalize(lr_batch[i])
        
        axes[i, 0].imshow(lr_img)
        axes[i, 0].set_title(f'LR ({lr_img.shape[0]}x{lr_img.shape[1]})')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(hr_img)
        axes[i, 1].set_title(f'HR ({hr_img.shape[0]}x{hr_img.shape[1]})')
        axes[i, 1].axis('off')
    
    plt.suptitle('Training Data Samples (LR / HR Pairs)', fontsize=14)
    plt.tight_layout()
    plt.show()
    print("\n✅ 数据加载验证成功！")
except Exception as e:
    print(f"❌ 数据可视化失败: {e}")

---
## 3. 模型架构

### 3.1 SRResNet 基线生成器
- 16个残差块
- 亚像素卷积（PixelShuffle）上采样
- 适合像素级损失训练

In [ ]:
# SRResNet 生成器
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor
from mindspore.common.initializer import Normal, HeNormal

class ResidualBlock(nn.Cell):
    """残差块：Conv-BN-PReLU-Conv-BN + Skip"""
    def __init__(self, channels=64):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, pad_mode='pad', 
                               has_bias=False, weight_init=HeNormal())
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, pad_mode='pad',
                               has_bias=False, weight_init=HeNormal())
        self.bn2 = nn.BatchNorm2d(channels)
    
    def construct(self, x):
        residual = x
        out = self.prelu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        return out + residual

class PixelShuffle(nn.Cell):
    """亚像素卷积上采样 - 与PyTorch兼容的手动实现
    
    将 (N, C*r*r, H, W) 变换为 (N, C, H*r, W*r)
    使用正确的通道重排顺序，避免棋盘格伪影
    """
    def __init__(self, scale):
        super(PixelShuffle, self).__init__()
        self.scale = scale
        self.reshape = ops.Reshape()
        self.transpose = ops.Transpose()
    
    def construct(self, x):
        n, c, h, w = x.shape
        r = self.scale
        out_c = c // (r * r)
        
        # (N, C*r*r, H, W) -> (N, out_c, r, r, H, W)
        x = self.reshape(x, (n, out_c, r, r, h, w))
        # (N, out_c, r, r, H, W) -> (N, out_c, H, r, W, r)
        x = self.transpose(x, (0, 1, 4, 2, 5, 3))
        # (N, out_c, H, r, W, r) -> (N, out_c, H*r, W*r)
        x = self.reshape(x, (n, out_c, h * r, w * r))
        
        return x

class UpsampleBlock(nn.Cell):
    """上采样块：Conv + PixelShuffle + PReLU"""
    def __init__(self, in_channels, scale=2):
        super(UpsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * (scale ** 2), 3, 
                              padding=1, pad_mode='pad', has_bias=False, weight_init=HeNormal())
        self.pixel_shuffle = PixelShuffle(scale)
        self.prelu = nn.PReLU()
    
    def construct(self, x):
        return self.prelu(self.pixel_shuffle(self.conv(x)))

class SRResNet(nn.Cell):
    """SRResNet 超分辨率生成器
    
    Args:
        scale: 放大倍数（2, 3, 4）
        n_residual: 残差块数量
        n_features: 特征通道数
    """
    def __init__(self, scale=4, n_residual=16, n_features=64):
        super(SRResNet, self).__init__()
        self.scale = scale
        
        # 输入卷积
        self.conv_input = nn.Conv2d(3, n_features, 9, padding=4, pad_mode='pad',
                                    has_bias=False, weight_init=HeNormal())
        self.prelu = nn.PReLU()
        
        # 残差块
        self.residual_blocks = nn.SequentialCell(
            [ResidualBlock(n_features) for _ in range(n_residual)]
        )
        
        # 残差后卷积
        self.conv_mid = nn.Conv2d(n_features, n_features, 3, padding=1, pad_mode='pad',
                                  has_bias=False, weight_init=HeNormal())
        self.bn_mid = nn.BatchNorm2d(n_features)
        
        # 上采样（×4 = 两次×2）
        upsample_layers = []
        if scale == 4:
            upsample_layers.append(UpsampleBlock(n_features, 2))
            upsample_layers.append(UpsampleBlock(n_features, 2))
        elif scale == 2:
            upsample_layers.append(UpsampleBlock(n_features, 2))
        elif scale == 3:
            upsample_layers.append(UpsampleBlock(n_features, 3))
        self.upsample = nn.SequentialCell(upsample_layers)
        
        # 输出卷积
        self.conv_output = nn.Conv2d(n_features, 3, 9, padding=4, pad_mode='pad',
                                     has_bias=False, weight_init=HeNormal())
        self.tanh = nn.Tanh()
    
    def construct(self, x):
        # 输入处理
        out = self.prelu(self.conv_input(x))
        residual = out
        
        # 残差块
        out = self.residual_blocks(out)
        out = self.bn_mid(self.conv_mid(out))
        out = out + residual
        
        # 上采样
        out = self.upsample(out)
        
        # 输出
        out = self.tanh(self.conv_output(out))
        return out

# 验证生成器
print("=== SRResNet 模型验证 ===")
generator = SRResNet(scale=SCALE, n_residual=16, n_features=64)
test_lr = Tensor(np.random.randn(1, 3, LR_SIZE, LR_SIZE).astype(np.float32))
test_sr = generator(test_lr)
print(f"输入 LR: {test_lr.shape}")
print(f"输出 SR: {test_sr.shape}")
print(f"期望 HR: (1, 3, {HR_SIZE}, {HR_SIZE})")
assert test_sr.shape == (1, 3, HR_SIZE, HR_SIZE), "输出尺寸不匹配！"
print("✅ SRResNet 构建成功！")

# 验证PixelShuffle正确性
print("\n=== PixelShuffle 验证 ===")
ps = PixelShuffle(2)
# 创建测试输入：每个r*r块内的值应该按正确顺序排列
test_input = Tensor(np.arange(16).reshape(1, 16, 1, 1).astype(np.float32))
test_output = ps(test_input)
print(f"输入shape: {test_input.shape} -> 输出shape: {test_output.shape}")
print(f"输出内容:\n{test_output.asnumpy()[0, 0]}")
# 正确的PixelShuffle应该输出:
# [[0, 1], [2, 3]] 对应第一个输出通道
print("✅ PixelShuffle 验证完成")

### 3.2 ESRGAN 判别器（PatchGAN）

In [ ]:
# ESRGAN 判别器（自适应输入尺寸）
class Discriminator(nn.Cell):
    """VGG风格判别器
    
    使用自适应平均池化，支持不同尺寸的HR输入
    """
    def __init__(self, in_channels=3, ndf=64):
        super(Discriminator, self).__init__()
        
        def conv_block(in_ch, out_ch, stride=1, bn=True):
            layers = [
                nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, 
                         pad_mode='pad', has_bias=not bn, weight_init=HeNormal())
            ]
            if bn:
                layers.append(nn.BatchNorm2d(out_ch))
            layers.append(nn.LeakyReLU(0.2))
            return nn.SequentialCell(layers)
        
        self.features = nn.SequentialCell([
            # 无BN的第一层
            conv_block(in_channels, ndf, stride=1, bn=False),
            conv_block(ndf, ndf, stride=2, bn=True),
            # H/2
            conv_block(ndf, ndf*2, stride=1, bn=True),
            conv_block(ndf*2, ndf*2, stride=2, bn=True),
            # H/4
            conv_block(ndf*2, ndf*4, stride=1, bn=True),
            conv_block(ndf*4, ndf*4, stride=2, bn=True),
            # H/8
            conv_block(ndf*4, ndf*8, stride=1, bn=True),
            conv_block(ndf*8, ndf*8, stride=2, bn=True),
            # H/16
        ])
        
        # 使用自适应平均池化，将任意尺寸特征图转换为固定的4x4
        self.adaptive_pool = ops.AdaptiveAvgPool2D((4, 4))
        
        # 分类头（输入固定为 ndf*8 * 4 * 4 = 512 * 16 = 8192）
        self.classifier = nn.SequentialCell([
            nn.Flatten(),
            nn.Dense(ndf*8 * 4 * 4, 1024),
            nn.LeakyReLU(0.2),
            nn.Dense(1024, 1),
            nn.Sigmoid()
        ])
    
    def construct(self, x):
        features = self.features(x)
        # 自适应池化到固定尺寸
        features = self.adaptive_pool(features)
        out = self.classifier(features)
        return out

# 验证判别器
print("\n=== Discriminator 模型验证 ===")
discriminator = Discriminator(in_channels=3, ndf=64)
test_hr = Tensor(np.random.randn(1, 3, HR_SIZE, HR_SIZE).astype(np.float32))
test_out = discriminator(test_hr)
print(f"输入 HR: {test_hr.shape}")
print(f"输出: {test_out.shape}")
print("✅ Discriminator 构建成功！")

---
## 4. 损失函数

- **像素损失（L1）**：保证像素级准确性
- **感知损失（VGG）**：保证感知质量（可选）
- **对抗损失（GAN）**：生成更真实的细节

In [ ]:
# 损失函数定义
class SRLoss(nn.Cell):
    """超分辨率综合损失
    
    L_total = λ_pixel * L_pixel + λ_adv * L_adv
    
    Args:
        lambda_pixel: 像素损失权重
        lambda_adv: 对抗损失权重
    """
    def __init__(self, lambda_pixel=1.0, lambda_adv=0.001, label_smoothing=0.1):
        super(SRLoss, self).__init__()
        self.lambda_pixel = lambda_pixel
        self.lambda_adv = lambda_adv
        self.label_smoothing = label_smoothing
        
        self.l1_loss = nn.L1Loss(reduction='mean')
        self.bce_loss = nn.BCELoss(reduction='mean')
    
    def pixel_loss(self, sr, hr):
        """像素级L1损失"""
        return self.l1_loss(sr, hr)
    
    def generator_loss(self, d_fake, sr, hr):
        """生成器总损失"""
        # 像素损失
        pixel_loss = self.l1_loss(sr, hr) * self.lambda_pixel
        
        # 对抗损失（生成器希望判别器输出接近1）
        real_label = Tensor(1.0 - self.label_smoothing, mindspore.float32)
        adv_loss = self.bce_loss(d_fake, ops.ones_like(d_fake) * real_label) * self.lambda_adv
        
        return pixel_loss + adv_loss, pixel_loss, adv_loss
    
    def discriminator_loss(self, d_real, d_fake):
        """判别器损失"""
        real_label = Tensor(1.0 - self.label_smoothing, mindspore.float32)
        fake_label = Tensor(self.label_smoothing, mindspore.float32)
        
        real_loss = self.bce_loss(d_real, ops.ones_like(d_real) * real_label)
        fake_loss = self.bce_loss(d_fake, ops.zeros_like(d_fake) + fake_label)
        
        return (real_loss + fake_loss) / 2

print("✅ 损失函数定义完成！")

---
## 5. 训练循环

In [ ]:
# 训练网络封装
from mindspore import save_checkpoint, load_checkpoint, load_param_into_net
from tqdm import tqdm
import time
from skimage.metrics import structural_similarity as ssim
import numpy as np

# 参数命名前缀（避免冲突）
def rename_params(net, prefix):
    for param in net.get_parameters():
        param.name = f"{prefix}_{param.name}"

def calculate_psnr(img1, img2):
    """计算PSNR（峰值信噪比）
    
    Args:
        img1, img2: uint8图像，shape为(H, W, C)
    Returns:
        PSNR值（dB）
    """
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    """计算SSIM（结构相似性）
    
    Args:
        img1, img2: uint8图像，shape为(H, W, C)
    Returns:
        SSIM值（0-1之间）
    """
    # 获取图像最小边长，动态调整win_size
    min_side = min(img1.shape[0], img1.shape[1])
    # win_size必须是奇数且不超过图像最小边长
    win_size = min(7, min_side)
    if win_size % 2 == 0:
        win_size -= 1
    if win_size < 3:
        win_size = 3
    return ssim(img1, img2, channel_axis=2, data_range=255, win_size=win_size)

class SRTrainer:
    """超分辨率训练器"""
    def __init__(self, generator, discriminator, loss_fn, 
                 lr_g=1e-4, lr_d=1e-4, beta1=0.9):
        self.G = generator
        self.D = discriminator
        self.loss_fn = loss_fn
        
        # 重命名参数
        rename_params(self.G, "G")
        rename_params(self.D, "D")
        
        # 优化器
        self.opt_G = nn.Adam(self.G.trainable_params(), learning_rate=lr_g, beta1=beta1)
        self.opt_D = nn.Adam(self.D.trainable_params(), learning_rate=lr_d, beta1=beta1)
        
        # 梯度函数
        self.grad_G = ops.value_and_grad(self._g_loss, None, self.opt_G.parameters)
        self.grad_D = ops.value_and_grad(self._d_loss, None, self.opt_D.parameters)
    
    def _g_loss(self, lr, hr):
        """生成器前向 + 损失"""
        sr = self.G(lr)
        d_fake = self.D(sr)
        total_loss, pixel_loss, adv_loss = self.loss_fn.generator_loss(d_fake, sr, hr)
        return total_loss
    
    def _d_loss(self, lr, hr):
        """判别器前向 + 损失"""
        sr = self.G(lr)
        sr = ops.stop_gradient(sr)  # 不计算生成器梯度
        d_real = self.D(hr)
        d_fake = self.D(sr)
        return self.loss_fn.discriminator_loss(d_real, d_fake)
    
    def train_step(self, lr, hr):
        """单步训练"""
        # 训练生成器
        g_loss, g_grads = self.grad_G(lr, hr)
        self.opt_G(g_grads)
        
        # 训练判别器
        d_loss, d_grads = self.grad_D(lr, hr)
        self.opt_D(d_grads)
        
        return g_loss, d_loss
    
    def generate(self, lr):
        """生成SR图像"""
        self.G.set_train(False)
        sr = self.G(lr)
        self.G.set_train(True)
        return sr

print("✅ 训练器定义完成！")
print("✅ PSNR/SSIM计算函数定义完成！")

In [ ]:
# 主训练循环配置
import os

# ===================== 两阶段训练配置说明 =====================
# 【方案二：两阶段完整训练】
# 
# 第一阶段（像素级预训练）：
#   - LAMBDA_ADV = 0.0（纯L1损失）
#   - EPOCHS = 600, LR_G = 2e-4
#   - n_residual = 23（增加模型容量，更好利用GPU）
#   - 预期PSNR: 25~27dB
#
# 第二阶段（GAN精调）- 完成第一阶段后取消注释并使用
# ============================================================

# ==================== 第一阶段配置 ====================
# EPOCHS = 600              # 训练轮数
# LR_G = 2e-4               # 生成器学习率
# LR_D = 1e-4               # 判别器学习率（未使用）
# LAMBDA_PIXEL = 1.0        # 像素损失权重
# LAMBDA_ADV = 0.0          # 对抗损失权重（第一阶段设为0）
# SAVE_DIR = "./ckpt_sr"    # 模型保存目录
# EVAL_INTERVAL = 25        # 评估间隔
# N_RESIDUAL = 23           # 残差块数量
# N_FEATURES = 64           # 特征通道数
# LR_DECAY = True
# LR_DECAY_EPOCHS = [300, 450, 550]
# LR_DECAY_FACTOR = 0.5
# PRETRAINED_G = None
# PRETRAINED_D = None
# ================================================================

# ==================== 第二阶段配置（优化版） ====================
# 优化点：
# 1. LAMBDA_ADV: 0.001 -> 0.005（增强GAN效果，提升纹理细节）
# 2. LR_D: 2e-5 -> 5e-5（与LR_G保持一致，确保D训练充分）
# 3. PRETRAINED_D: 支持从检查点恢复D模型
EPOCHS = 200
LR_G = 5e-5               # 生成器学习率（精调用小学习率）
LR_D = 5e-5               # 判别器学习率（与G保持一致）
LAMBDA_PIXEL = 1.0        # 像素损失权重
LAMBDA_ADV = 0.005        # 对抗损失权重（优化：0.001->0.005）
SAVE_DIR = "./ckpt_sr_stage2"
EVAL_INTERVAL = 20
N_RESIDUAL = 23           # 必须与第一阶段一致
N_FEATURES = 64           # 必须与第一阶段一致
LR_DECAY = True
LR_DECAY_EPOCHS = [100, 150, 180]
LR_DECAY_FACTOR = 0.5
PRETRAINED_G = "./ckpt_sr/G_best.ckpt"  # 加载第一阶段最佳模型
PRETRAINED_D = None  # 首次GAN训练设为None，恢复训练时设为 "./ckpt_sr_stage2/D_best.ckpt"
# ================================================================

os.makedirs(SAVE_DIR, exist_ok=True)

# 初始化模型
print("=" * 60)
print("初始化模型...")
print("=" * 60)

G = SRResNet(scale=SCALE, n_residual=N_RESIDUAL, n_features=N_FEATURES)
D = Discriminator(in_channels=3, ndf=64)

# 重命名参数（必须在加载权重之前）
rename_params(G, "G")
rename_params(D, "D")

# 加载预训练权重（第二阶段使用）
if PRETRAINED_G is not None and os.path.exists(PRETRAINED_G):
    param_dict = load_checkpoint(PRETRAINED_G)
    load_param_into_net(G, param_dict)
    print(f"✅ 已加载生成器预训练权重: {PRETRAINED_G}")

if PRETRAINED_D is not None and os.path.exists(PRETRAINED_D):
    param_dict = load_checkpoint(PRETRAINED_D)
    load_param_into_net(D, param_dict)
    print(f"✅ 已加载判别器预训练权重: {PRETRAINED_D}")

# 统计参数量
g_params = sum(p.size for p in G.get_parameters()) / 1e6
d_params = sum(p.size for p in D.get_parameters()) / 1e6
print(f"生成器参数量: {g_params:.2f}M")
print(f"判别器参数量: {d_params:.2f}M")

# 训练形式
print(f"\n训练形式: {'GAN (对抗训练)' if LAMBDA_ADV > 0 else '纯像素损失'}")
print(f"模型配置: n_residual={N_RESIDUAL}, n_features={N_FEATURES}")
print(f"LR shape: (1, 3, {LR_SIZE}, {LR_SIZE})")

# 创建损失函数和优化器
loss_fn = SRLoss(lambda_pixel=LAMBDA_PIXEL, lambda_adv=LAMBDA_ADV)
optimizer_G = nn.Adam(G.trainable_params(), learning_rate=LR_G, beta1=0.9)
optimizer_D = nn.Adam(D.trainable_params(), learning_rate=LR_D, beta1=0.9)

print(f"\n=== 开始训练 (共 {EPOCHS} 轮) ===")
print(f"配置: LR_G={LR_G}, LR_D={LR_D}, λ_pixel={LAMBDA_PIXEL}, λ_adv={LAMBDA_ADV}")
if LR_DECAY:
    print(f"学习率衰减: 在第{LR_DECAY_EPOCHS}轮，衰减因子={LR_DECAY_FACTOR}")

# 定义训练步骤
def _g_loss(lr, hr):
    sr = G(lr)
    d_fake = D(sr)
    total_loss, pixel_loss, adv_loss = loss_fn.generator_loss(d_fake, sr, hr)
    return total_loss

def _d_loss(lr, hr):
    sr = G(lr)
    sr = ops.stop_gradient(sr)
    d_real = D(hr)
    d_fake = D(sr)
    return loss_fn.discriminator_loss(d_real, d_fake)

def _pixel_loss(lr, hr):
    sr = G(lr)
    return loss_fn.pixel_loss(sr, hr)

grad_G = ops.value_and_grad(_g_loss, None, optimizer_G.parameters)
grad_D = ops.value_and_grad(_d_loss, None, optimizer_D.parameters)
grad_pixel = ops.value_and_grad(_pixel_loss, None, optimizer_G.parameters)

def train_step_gan(lr, hr):
    """GAN训练步骤"""
    g_loss, g_grads = grad_G(lr, hr)
    optimizer_G(g_grads)
    d_loss, d_grads = grad_D(lr, hr)
    optimizer_D(d_grads)
    return g_loss, d_loss

def train_step_pixel(lr, hr):
    """纯像素损失训练步骤"""
    loss, grads = grad_pixel(lr, hr)
    optimizer_G(grads)
    return loss

In [ ]:
# 训练循环
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 记录训练历史
history = {
    'epoch': [],
    'g_loss': [],
    'd_loss': [],
    'psnr': [],
    'ssim': []
}

best_psnr = 0.0

# 获取固定的验证样本用于可视化
fixed_sample = next(val_dataset.create_dict_iterator())
fixed_lr = fixed_sample['lr']
fixed_hr = fixed_sample['hr'].asnumpy()[0]

print(f"开始训练...")
print(f"训练集大小: {train_dataset.get_dataset_size()} batches")
print(f"验证集大小: {val_dataset.get_dataset_size()} batches")
print("=" * 60)

for epoch in range(1, EPOCHS + 1):
    G.set_train(True)
    if LAMBDA_ADV > 0:
        D.set_train(True)
    
    epoch_g_loss = 0.0
    epoch_d_loss = 0.0
    n_batches = 0
    
    # 训练进度条
    train_pbar = tqdm(train_dataset.create_dict_iterator(), 
                     total=train_dataset.get_dataset_size(),
                     desc=f"Epoch {epoch:3d}/{EPOCHS}",
                     leave=False)
    
    for batch in train_pbar:
        hr = batch['hr']
        lr = batch['lr']
        
        # GAN训练
        if LAMBDA_ADV > 0:
            g_loss, d_loss = train_step_gan(lr, hr)
            epoch_d_loss += d_loss.asnumpy()
        else:
            g_loss = train_step_pixel(lr, hr)
        
        epoch_g_loss += g_loss.asnumpy()
        n_batches += 1
        
        # 更新进度条
        if LAMBDA_ADV > 0:
            train_pbar.set_postfix({
                'G_loss': f"{g_loss.asnumpy():.4f}",
                'D_loss': f"{d_loss.asnumpy():.4f}"
            })
        else:
            train_pbar.set_postfix({'loss': f"{g_loss.asnumpy():.4f}"})
    
    avg_g_loss = epoch_g_loss / n_batches
    avg_d_loss = epoch_d_loss / n_batches if LAMBDA_ADV > 0 else 0
    
    # 学习率衰减
    if LR_DECAY and epoch in LR_DECAY_EPOCHS:
        current_lr_g = float(optimizer_G.learning_rate.data.asnumpy())
        new_lr_g = current_lr_g * LR_DECAY_FACTOR
        optimizer_G.learning_rate.set_data(Tensor(new_lr_g, mindspore.float32))
        tqdm.write(f"   📉 学习率衰减: G {current_lr_g:.2e} -> {new_lr_g:.2e}")
        
        if LAMBDA_ADV > 0:
            current_lr_d = float(optimizer_D.learning_rate.data.asnumpy())
            new_lr_d = current_lr_d * LR_DECAY_FACTOR
            optimizer_D.learning_rate.set_data(Tensor(new_lr_d, mindspore.float32))
            tqdm.write(f"   📉 学习率衰减: D {current_lr_d:.2e} -> {new_lr_d:.2e}")
    
    # 验证
    if epoch % EVAL_INTERVAL == 0 or epoch == EPOCHS:
        G.set_train(False)
        psnr_list = []
        ssim_list = []
        
        for val_batch in val_dataset.create_dict_iterator():
            val_lr = val_batch['lr']
            val_hr = val_batch['hr']
            
            sr = G(val_lr)
            
            # 计算指标
            sr_np = denormalize(sr.asnumpy()[0])
            hr_np = denormalize(val_hr.asnumpy()[0])
            
            psnr_val = calculate_psnr(sr_np, hr_np)
            ssim_val = calculate_ssim(sr_np, hr_np)
            psnr_list.append(psnr_val)
            ssim_list.append(ssim_val)
        
        avg_psnr = np.mean(psnr_list)
        avg_ssim = np.mean(ssim_list)
        
        history['epoch'].append(epoch)
        history['g_loss'].append(avg_g_loss)
        history['d_loss'].append(avg_d_loss)
        history['psnr'].append(avg_psnr)
        history['ssim'].append(avg_ssim)
        
        # 打印日志
        if LAMBDA_ADV > 0:
            tqdm.write(f"\nEpoch {epoch:3d}/{EPOCHS} | Loss: {avg_g_loss:.4f} | "
                  f"PSNR: {avg_psnr:.2f}dB | SSIM: {avg_ssim:.4f} | "
                  f"G_loss: {avg_g_loss:.4f} | D_loss: {avg_d_loss:.4f}")
        else:
            tqdm.write(f"\nEpoch {epoch:3d}/{EPOCHS} | Loss: {avg_g_loss:.4f} | "
                  f"PSNR: {avg_psnr:.2f}dB | SSIM: {avg_ssim:.4f}")
        
        # 保存最佳模型（同时保存G和D）
        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            save_checkpoint(G, os.path.join(SAVE_DIR, "G_best.ckpt"))
            if LAMBDA_ADV > 0:
                save_checkpoint(D, os.path.join(SAVE_DIR, "D_best.ckpt"))
                tqdm.write(f"   ✅ 保存最佳模型 G+D (PSNR: {best_psnr:.2f}dB)")
            else:
                tqdm.write(f"   ✅ 保存最佳模型 (PSNR: {best_psnr:.2f}dB)")
        
        if fixed_lr is not None:
            G.set_train(False)
            sr = G(fixed_lr).asnumpy()[0]
            G.set_train(True)
            
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            lr_img = denormalize(fixed_lr.asnumpy()[0])
            axes[0].imshow(lr_img)
            axes[0].set_title(f'LR Input ({lr_img.shape[0]}x{lr_img.shape[1]})')
            axes[0].axis('off')
            
            sr_img = denormalize(sr)
            axes[1].imshow(sr_img)
            axes[1].set_title(f'SR Output ({sr_img.shape[0]}x{sr_img.shape[1]})')
            axes[1].axis('off')
            
            hr_img = denormalize(fixed_hr)
            axes[2].imshow(hr_img)
            axes[2].set_title(f'HR Ground Truth ({hr_img.shape[0]}x{hr_img.shape[1]})')
            axes[2].axis('off')
            
            plt.suptitle(f'Epoch {epoch} - PSNR: {avg_psnr:.2f}dB, SSIM: {avg_ssim:.4f}', fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(SAVE_DIR, f'result_epoch_{epoch}.png'), dpi=150)
            plt.show()
    
    # 定期保存检查点
    if epoch % EVAL_INTERVAL == 0:
        save_checkpoint(G, os.path.join(SAVE_DIR, f"G_epoch_{epoch}.ckpt"))
        if LAMBDA_ADV > 0:
            save_checkpoint(D, os.path.join(SAVE_DIR, f"D_epoch_{epoch}.ckpt"))

# 保存最终模型
save_checkpoint(G, os.path.join(SAVE_DIR, "G_final.ckpt"))
if LAMBDA_ADV > 0:
    save_checkpoint(D, os.path.join(SAVE_DIR, "D_final.ckpt"))

print(f"\n🎉 训练完成！最佳PSNR: {best_psnr:.2f}dB")
print(f"模型保存到: {SAVE_DIR}")

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 损失曲线
axes[0].plot(history['g_loss'], label='Generator Loss', color='blue')
axes[0].plot(history['d_loss'], label='Discriminator Loss', color='red')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# PSNR曲线
eval_epochs = list(range(EVAL_INTERVAL, len(history['psnr'])*EVAL_INTERVAL+1, EVAL_INTERVAL))
if len(eval_epochs) > len(history['psnr']):
    eval_epochs = eval_epochs[:len(history['psnr'])]
axes[1].plot(eval_epochs, history['psnr'], marker='o', color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('PSNR (dB)')
axes[1].set_title('PSNR on Validation Set')
axes[1].grid(True, alpha=0.3)

# SSIM曲线
axes[2].plot(eval_epochs, history['ssim'], marker='s', color='orange')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('SSIM')
axes[2].set_title('SSIM on Validation Set')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'training_curves.png'), dpi=150)
plt.show()

---
## 6. 模型评估

In [ ]:
# 加载最佳模型进行评估
print("=== 加载最佳模型进行评估 ===")

G_eval = SRResNet(scale=SCALE, n_residual=N_RESIDUAL, n_features=N_FEATURES)
rename_params(G_eval, "G")

ckpt_path = os.path.join(SAVE_DIR, "G_best.ckpt")
if os.path.exists(ckpt_path):
    param_dict = load_checkpoint(ckpt_path)
    load_param_into_net(G_eval, param_dict)
    print(f"✅ 已加载模型: {ckpt_path}")
else:
    print(f"⚠️ 未找到最佳模型，使用当前模型")
    G_eval = G

G_eval.set_train(False)

# 在验证集上评估
print("\n[验证集评估]")
psnr_list = []
ssim_list = []

for batch in val_dataset.create_dict_iterator():
    hr = batch['hr'].asnumpy()
    lr = batch['lr']
    sr = G_eval(lr).asnumpy()
    
    for i in range(sr.shape[0]):
        # 修复：使用 denormalize 转换格式 (CHW, [-1,1]) -> (HWC, [0,255] uint8)
        sr_img = denormalize(sr[i])
        hr_img = denormalize(hr[i])
        
        p = calculate_psnr(sr_img, hr_img)
        s = calculate_ssim(sr_img, hr_img)
        psnr_list.append(p)
        ssim_list.append(s)
        print(f"  样本 {len(psnr_list)}: PSNR={p:.2f}dB, SSIM={s:.4f}")

print(f"\n平均 PSNR: {np.mean(psnr_list):.2f}dB")
print(f"平均 SSIM: {np.mean(ssim_list):.4f}")

# 验收标准判断
print("\n[验收标准检查]")
if np.mean(psnr_list) >= 28:
    print(f"✅ PSNR ≥ 28dB (实际: {np.mean(psnr_list):.2f}dB)")
else:
    print(f"❌ PSNR < 28dB (实际: {np.mean(psnr_list):.2f}dB)，建议增加训练轮数")

if np.mean(ssim_list) >= 0.80:
    print(f"✅ SSIM ≥ 0.80 (实际: {np.mean(ssim_list):.4f})")
else:
    print(f"❌ SSIM < 0.80 (实际: {np.mean(ssim_list):.4f})，建议调整损失权重")

---
## 7. 推理与可视化

对真实旧照片进行修复增强

In [ ]:
# 旧照片推理（优化版：显存释放 + 大图分块处理）
from PIL import Image
import os
import gc

# ====== 关键：释放训练占用的显存 ======
print("=== 释放训练占用的显存 ===")

# 删除训练时的大对象
vars_to_delete = ['G', 'D', 'trainer', 'train_dataset', 'val_dataset', 
                  'fixed_lr', 'fixed_hr', 'history']
for var_name in vars_to_delete:
    if var_name in dir():
        try:
            exec(f"del {var_name}")
            print(f"  已删除: {var_name}")
        except:
            pass

# 强制Python垃圾回收
gc.collect()

# MindSpore 显存清理
try:
    from mindspore import context
    # 重置计算图缓存
    context.set_context(reserve_class_name_in_scope=False)
    print("✅ 显存清理完成")
except Exception as e:
    print(f"⚠️ 显存清理警告: {e}")

# ====== 配置 ======
OLD_PHOTOS_DIR = os.path.join(DATA_ROOT, 'old_photos')
OUTPUT_DIR = "./output_sr"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 大图处理参数
MAX_INPUT_SIZE = 256  # 最大输入尺寸（超过则缩放）
TILE_SIZE = 128       # 分块大小（显存不足时使用）
TILE_OVERLAP = 16     # 分块重叠（避免接缝）

def inference_single_safe(model, image_path, scale=4, max_size=256):
    """安全的单张图像推理（支持大图缩放）
    
    Args:
        model: 生成器模型
        image_path: 图像路径
        scale: 放大倍数
        max_size: 最大输入尺寸，超过则先缩放
    """
    # 读取图像
    img = Image.open(image_path).convert('RGB')
    original_size = img.size  # (W, H)
    print(f"   原始尺寸: {original_size[0]}x{original_size[1]}")
    
    # 如果图像太大，先缩放到max_size
    w, h = original_size
    if max(w, h) > max_size:
        ratio = max_size / max(w, h)
        new_w = int(w * ratio)
        new_h = int(h * ratio)
        img_input = img.resize((new_w, new_h), Image.BICUBIC)
        print(f"   缩放至: {new_w}x{new_h} (显存优化)")
    else:
        img_input = img
    
    # 预处理
    img_np = np.array(img_input).astype(np.float32) / 127.5 - 1.0
    img_np = img_np.transpose(2, 0, 1)  # HWC -> CHW
    img_tensor = Tensor(img_np[np.newaxis, ...], mindspore.float32)
    
    # 推理
    model.set_train(False)
    sr_tensor = model(img_tensor)
    
    # 后处理
    sr_np = sr_tensor.asnumpy()[0]
    sr_np = (sr_np * 127.5 + 127.5).clip(0, 255).astype(np.uint8)
    sr_np = sr_np.transpose(1, 2, 0)  # CHW -> HWC
    sr_img = Image.fromarray(sr_np)
    
    return img, sr_img

def inference_tiled(model, image_path, scale=4, tile_size=128, overlap=16):
    """分块推理（适用于超大图像）
    
    将大图切分成小块分别处理，最后拼接
    """
    img = Image.open(image_path).convert('RGB')
    original_size = img.size
    w, h = original_size
    print(f"   原始尺寸: {w}x{h}")
    print(f"   使用分块处理: tile={tile_size}, overlap={overlap}")
    
    # 输出图像尺寸
    out_w, out_h = w * scale, h * scale
    sr_result = np.zeros((out_h, out_w, 3), dtype=np.float32)
    sr_count = np.zeros((out_h, out_w, 1), dtype=np.float32)
    
    # 分块处理
    stride = tile_size - overlap
    tiles_x = (w + stride - 1) // stride
    tiles_y = (h + stride - 1) // stride
    total_tiles = tiles_x * tiles_y
    
    tile_idx = 0
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            tile_idx += 1
            # 计算实际tile区域
            x1, y1 = x, y
            x2 = min(x + tile_size, w)
            y2 = min(y + tile_size, h)
            
            # 裁剪tile
            tile = img.crop((x1, y1, x2, y2))
            tile_w, tile_h = tile.size
            
            # 预处理
            tile_np = np.array(tile).astype(np.float32) / 127.5 - 1.0
            tile_np = tile_np.transpose(2, 0, 1)
            tile_tensor = Tensor(tile_np[np.newaxis, ...], mindspore.float32)
            
            # 推理
            model.set_train(False)
            sr_tile = model(tile_tensor).asnumpy()[0]
            sr_tile = sr_tile.transpose(1, 2, 0)  # CHW -> HWC
            
            # 放置到输出位置
            ox1, oy1 = x1 * scale, y1 * scale
            ox2, oy2 = ox1 + tile_w * scale, oy1 + tile_h * scale
            
            sr_result[oy1:oy2, ox1:ox2] += sr_tile
            sr_count[oy1:oy2, ox1:ox2] += 1
            
            if tile_idx % 10 == 0:
                print(f"   进度: {tile_idx}/{total_tiles}")
    
    # 平均重叠区域
    sr_result = sr_result / np.maximum(sr_count, 1)
    sr_result = (sr_result * 127.5 + 127.5).clip(0, 255).astype(np.uint8)
    sr_img = Image.fromarray(sr_result)
    
    return img, sr_img

# ====== 重新加载推理模型（轻量级） ======
print("\n=== 重新加载推理模型 ===")
G_eval = SRResNet(scale=SCALE, n_residual=16, n_features=64)
rename_params(G_eval, "G")

ckpt_path = os.path.join(SAVE_DIR, "G_best.ckpt")
if os.path.exists(ckpt_path):
    param_dict = load_checkpoint(ckpt_path)
    load_param_into_net(G_eval, param_dict)
    print(f"✅ 已加载模型: {ckpt_path}")
else:
    print(f"❌ 未找到模型: {ckpt_path}")
    raise FileNotFoundError(f"请确保模型文件存在: {ckpt_path}")

G_eval.set_train(False)

# ====== 处理所有旧照片 ======
print("\n=== 旧照片修复推理 ===")
old_photos = [f for f in os.listdir(OLD_PHOTOS_DIR) if f.lower().endswith(IMAGE_EXTENSIONS)]
print(f"找到 {len(old_photos)} 张旧照片")

for photo_name in old_photos:
    photo_path = os.path.join(OLD_PHOTOS_DIR, photo_name)
    print(f"\n处理: {photo_name}")
    
    try:
        # 检查图像尺寸决定处理方式
        with Image.open(photo_path) as tmp_img:
            w, h = tmp_img.size
        
        if max(w, h) > 512:
            # 超大图：使用分块处理
            print(f"   [分块模式] 图像较大 ({w}x{h})")
            original, restored = inference_tiled(G_eval, photo_path, SCALE, 
                                                  tile_size=TILE_SIZE, overlap=TILE_OVERLAP)
        elif max(w, h) > MAX_INPUT_SIZE:
            # 大图：缩放后处理
            print(f"   [缩放模式] 图像中等 ({w}x{h})")
            original, restored = inference_single_safe(G_eval, photo_path, SCALE, MAX_INPUT_SIZE)
        else:
            # 小图：直接处理
            print(f"   [直接模式] 图像较小 ({w}x{h})")
            original, restored = inference_single_safe(G_eval, photo_path, SCALE, max_size=w)
        
        # 保存结果
        stem = os.path.splitext(photo_name)[0]
        restored.save(os.path.join(OUTPUT_DIR, f"{stem}_restored.png"))
        
        # 可视化对比
        fig, axes = plt.subplots(1, 2, figsize=(14, 7))
        
        axes[0].imshow(original)
        axes[0].set_title(f'Original ({original.size[0]}x{original.size[1]})', fontsize=12)
        axes[0].axis('off')
        
        axes[1].imshow(restored)
        axes[1].set_title(f'Restored ({restored.size[0]}x{restored.size[1]})', fontsize=12)
        axes[1].axis('off')
        
        plt.suptitle(f'Old Photo Restoration: {photo_name}', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, f"{stem}_comparison.png"), dpi=150)
        plt.show()
        
        print(f"   ✅ 保存到: {OUTPUT_DIR}/{stem}_restored.png")
        
        # 每张图后清理一次
        gc.collect()
        
    except Exception as e:
        print(f"   ❌ 处理失败: {e}")
        import traceback
        traceback.print_exc()

print(f"\n🎉 推理完成！结果保存在: {OUTPUT_DIR}")

---
## 8. 项目总结

### 完成内容
1. ✅ 环境搭建与验证（MindSpore 2.2.14）
2. ✅ 数据集加载（LR/HR配对，支持随机裁剪和增强）
3. ✅ SRResNet基线生成器（16残差块 + PixelShuffle上采样）
4. ✅ ESRGAN判别器（VGG风格）
5. ✅ 损失函数（像素L1 + 对抗损失）
6. ✅ 训练循环（支持PSNR/SSIM评估）
7. ✅ 模型保存与加载
8. ✅ 旧照片推理与可视化

### 评估指标
- PSNR: 越高越好（目标 ≥28dB）
- SSIM: 越高越好（目标 ≥0.80）

### 优化建议
1. 增加训练数据量（推荐 ≥800对）
2. 添加感知损失（VGG特征）
3. 使用RRDB替代ResBlock（ESRGAN）
4. 数据增强：旋转、缩放、颜色抖动

In [ ]:
print("=" * 60)
print("高分辨率图像修复与增强项目 - 完成")
print("=" * 60)
print(f"\n模型保存路径: {SAVE_DIR}")
print(f"推理结果路径: {OUTPUT_DIR}")
print("\n项目文件:")
print("  - G_best.ckpt: 最佳生成器权重")
print("  - G_final.ckpt: 最终生成器权重")
print("  - training_curves.png: 训练曲线")
print("  - result_epoch_*.png: 各轮结果")